In [1]:
import matplotlib.pyplot as plt, numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

2022-12-15 08:12:36.519132: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-15 08:12:36.695551: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-15 08:12:37.455189: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2022-12-15 08:12:37.455302: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'li

In [3]:
total_ds = tf.data.experimental.make_csv_dataset(
    "daphnet/data.csv",
    batch_size=32, # Artificially small to make examples easier to show.
    column_names = [f"input_{i+1}" for i in range(769)],
    label_name="input_769",
    num_epochs=1, header=False,
    ignore_errors=True)

In [4]:
DATASET_SIZE = 1140835
train_size = int(0.7 * DATASET_SIZE)
val_size = int(0.15 * DATASET_SIZE)
test_size = int(0.15 * DATASET_SIZE)

train_ds = total_ds.take(train_size)
test_ds = total_ds.skip(train_size)
val_ds = test_ds.skip(test_size)
test_ds = test_ds.take(test_size)

In [5]:
input_list = []
for column in range(768):
    _input = tf.keras.Input(shape=(1,))
    input_list.append(_input)

concat = tf.keras.layers.Concatenate(name="concat")(input_list)
dense = tf.keras.layers.Dense(256, activation="relu", name="dense", dtype='float64' )(concat)
output_dense = tf.keras.layers.Dense(1, activation="sigmoid", name="labeling", dtype='float64')(dense)
model = tf.keras.Model(inputs=input_list, outputs=output_dense)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])


In [6]:
input_shape = 768

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=callbacks,
)

Epoch 1/10
  35652/Unknown - 1694s 47ms/step - loss: 2.9738 - accuracy: 0.8904 - precision: 0.4353 - recall: 0.4332WARNING:tensorflow:Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy,precision,recall
35652/35652 [==============================] - 3192s 89ms/step - loss: 2.9738 - accuracy: 0.8904 - precision: 0.4353 - recall: 0.4332
Epoch 2/10
21971/35652 [=================>............] - ETA: 13:47 - loss: 0.1842 - accuracy: 0.9169 - precision: 0.6617 - recall: 0.4610

KeyboardInterrupt: 